## imports

In [6]:
import requests
import os
from main import list_files_in_folder
from dotenv import load_dotenv
import json
import pandas as pd
load_dotenv()
deepinfra_api_key = os.getenv("DEEPINFRA_KEY")

In [3]:

def get_chat_response(prompt, api_key=deepinfra_api_key):
    url = "https://api.deepinfra.com/v1/openai/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    data = {
        "model": "meta-llama/Meta-Llama-3.1-70B-Instruct",
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ]
    }
    
    response = requests.post(url, headers=headers, json=data)
    
    if response.status_code == 200:
        response_json = response.json()
        return response_json["choices"][0]["message"]["content"]
    else:
        return f"Error: {response.status_code} - {response.text}"

# Example usage
prompt = "Hello!"
response = get_chat_response(prompt)
print(response)


Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?


## Load Data

In [17]:
with open('nogit/prompts_cea_HTV_200_80perc_nofuzzy.json') as f:
    inputs = json.load(f)

gt_path = 'data/HardTablesR1/DataSets/HardTablesR1/Valid/gt/cea_gt.csv'

gt = pd.read_csv(gt_path, header=None)

In [20]:
p_desc = inputs['ZRWO683W']['(1, 0)']['cea_prompt_desc']
p_nodesc = inputs['ZRWO683W']['(1, 0)']['cea_prompt_nodesc']

In [23]:
with open('results/outputsCEA3.json') as f:
    outs = json.load(f)

## Inference


In [21]:
out_desc = get_chat_response(p_desc)

In [29]:
print(out_desc)
print(outs['ZRWO683W']['(1, 0)']['output_desc'])
print(outs['ZRWO683W']['(1, 0)']['output_nodesc'])
outs['ZRWO683W']['(1, 0)'].keys()


[[Q52820308]]
[[[Q17321167]]]
{'id': 'Q17321158'}


dict_keys(['cell', 'table_desc', 'cea_prompt_desc', 'cea_prompt_nodesc', 'output_desc', 'output_nodesc'])

In [41]:
tot = 0
for k, v in inputs.items():
    tot += len(v)  


In [42]:
outputs = inputs.copy()

In [43]:
from tqdm import tqdm
for t_name, t_dict in tqdm(inputs.items()):
    for k, v in t_dict.items():
        p_desc = v['cea_prompt_desc']
        p_nodesc = v['cea_prompt_nodesc']
        out_desc = get_chat_response(p_desc)
        out_nodesc = get_chat_response(p_nodesc)
        outputs[t_name][k]['output_desc'] = out_desc
        outputs[t_name][k]['output_nodesc'] = out_nodesc

100%|██████████| 200/200 [3:46:42<00:00, 68.01s/it]   


In [44]:
with open('results/outputsCEA_LLAMA3.json', 'w') as f:
    json.dump(outputs, f)